In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import sys
from multiprocessing import Process
from tensorflow import keras
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

In [3]:
!wget https://storage.googleapis.com/cse144/tiny-imagenet-100.zip
!unzip -q tiny-imagenet-100.zip

--2022-03-20 01:52:57--  https://storage.googleapis.com/cse144/tiny-imagenet-100.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 74.125.199.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113846330 (109M) [application/zip]
Saving to: ‘tiny-imagenet-100.zip’

tiny-imagenet-100.z 100%[===================>] 108.57M   131MB/s    in 0.8s    

2022-03-20 01:52:58 (131 MB/s) - ‘tiny-imagenet-100.zip’ saved [113846330/113846330]



In [4]:
from PIL import Image
from glob import glob

In [5]:
label_dict = {}
for i, line in enumerate(open("tiny-imagenet-100/wnids.txt", "r")):
    label_dict[line.rstrip("\n")] = i

In [6]:
images = glob("tiny-imagenet-100/train/*/*/*")
trainingData = []
trainingLabel = []
for i in images:
    img = np.array(Image.open(i).convert("RGB"))
    label = label_dict[i.split('/')[-1].split('_')[0]]
    trainingLabel.append(label)
    trainingData.append(img)
    samples = expand_dims(img, 0)
    datagen = ImageDataGenerator(rotation_range=90)
    it = datagen.flow(samples, batch_size=1)
    for i in range(3):
      batch = it.next()
      image = batch[0].astype('uint8')
      trainingData.append(image)
      trainingLabel.append(label)


In [7]:
trainingData = np.array(trainingData)
trainingLabel = np.array(trainingLabel)
x_train, x_val, y_train, y_val = train_test_split(trainingData, trainingLabel, test_size = 0.10, shuffle = True)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.10, shuffle = True)
# def preprocess(x, y):
#   x = tf.cast(x, tf.float32) / 255.0
#   y = tf.cast(y, tf.int64)

#   return x, y
# x_train,y_train = preprocess(x_train, y_train)
# x_val, y_val = preprocess(x_val, y_val)

In [8]:
resnet_model = keras.Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(64,64,3),
                   pooling='avg',classes=100,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

94781440/94765736 [==============================] - 1s 0us/step


In [9]:
resnet_model.add(keras.layers.Flatten())
resnet_model.add(keras.layers.Dense(units = 1024, activation = 'relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(512, activation='relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(256, activation='relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(128, activation='relu'))
resnet_model.add(keras.layers.BatchNormalization())
resnet_model.add(keras.layers.Dropout(0.5))
resnet_model.add(keras.layers.Dense(100, activation = 'softmax'))

In [10]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                        

In [11]:
reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, mode='auto')
early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, mode='auto')
opt = keras.optimizers.SGD(learning_rate=0.1)
resnet_model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = resnet_model.fit(x_train, y_train, 
                           validation_data=(x_val, y_val), 
                           epochs=20
                          )

Epoch 1/20
5063/5063 [==============================] - 110s 19ms/step - loss: 3.5283 - accuracy: 0.1558 - val_loss: 2.8361 - val_accuracy: 0.2941
Epoch 2/20
5063/5063 [==============================] - 98s 19ms/step - loss: 3.1270 - accuracy: 0.2341 - val_loss: 2.6603 - val_accuracy: 0.3318
Epoch 3/20
5063/5063 [==============================] - 100s 20ms/step - loss: 3.0042 - accuracy: 0.2624 - val_loss: 2.5601 - val_accuracy: 0.3577
Epoch 4/20
5063/5063 [==============================] - 99s 20ms/step - loss: 2.9321 - accuracy: 0.2811 - val_loss: 2.5241 - val_accuracy: 0.3650
Epoch 5/20
5063/5063 [==============================] - 98s 19ms/step - loss: 2.8799 - accuracy: 0.2939 - val_loss: 2.4851 - val_accuracy: 0.3776
Epoch 6/20
5063/5063 [==============================] - 100s 20ms/step - loss: 2.8449 - accuracy: 0.3025 - val_loss: 2.4552 - val_accuracy: 0.3839
Epoch 7/20
5063/5063 [==============================] - 97s 19ms/step - loss: 2.8121 - accuracy: 0.3113 - val_loss: 2.435

In [12]:
test_loss, test_acc = resnet_model.evaluate(x_test, y_test)
print(test_loss, test_acc)

563/563 [==============================] - 13s 21ms/step - loss: 2.2920 - accuracy: 0.4246
2.2919726371765137 0.42455556988716125
